In [18]:
import ee
from itertools import product

# Initialize Earth Engine
ee.Authenticate()

ee.Initialize()

# Constants
start_date = '2003-01-01'
end_date = '2025-01-01'
year0 = '2007'

# Geometry
okavango = ee.Geometry.Polygon([
    [[21.696823393895034, -18.221176298743337],
     [21.696823393895034, -20.223198626985177],
     [24.102829253270034, -20.223198626985177],
     [24.102829253270034, -18.221176298743337]]
])

# Load DSWE ImageCollection
assets_dswe = ee.data.listAssets('projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m')['assets']

assets_dswe = list(filter(lambda asset: asset['type'] == 'IMAGE', assets_dswe))


asset_ids_dswe = [a['id'] for a in assets_dswe]

def process_dswe_image(id):
    year = ee.String(id).split('_').get(-2)
    month = ee.String(id).split('_').get(-1)
    img = ee.Image(id)
    binary = img.gt(0).toFloat().rename('dswe_bin')
    return img.addBands(binary).set({'year': year, 'month': month, 'date': ee.String(year).cat('-').cat(month)})

dswe_collection = ee.ImageCollection([process_dswe_image(id) for id in asset_ids_dswe])
print (dswe_collection.first().getInfo())
# Filters for year-specific and seasonal averages
dswe_compare = dswe_collection \
    .filter(ee.Filter.lt('month', '10')) \
    .filter(ee.Filter.gt('month', '06')) \
    .filter(ee.Filter.lt('year', '2025')) \


dswe_1990s = dswe_collection \
    .filter(ee.Filter.gt('month', '07')) \
    .filter(ee.Filter.lt('month', '10')) \
    .filter(ee.Filter.gte('year', '1985')) \
    .filter(ee.Filter.lte('year', '2005')) \
    .mean()

dswe_2010s = dswe_collection \
    .filter(ee.Filter.gt('month', '07')) \
    .filter(ee.Filter.lt('month', '10')) \
    .filter(ee.Filter.gte('year', '2020')) \
    .filter(ee.Filter.lte('year', '2025')) \
    .mean()

dswe_change = dswe_2010s.subtract(dswe_1990s)

visParamsDSWE = { 'palette' : ['white', '#deebf7','#9ecae1','#3182bd'], 'min' : 0,  'max': 1}

visParams = {'palette' : ['#a6611a','#dfc27d','#f5f5f5','#80cdc1','#018571'], 'min': -1., 'max' : 1}

task_change = ee.batch.Export.image.toDrive(
    image=dswe_change.select('dswe').unmask(0).updateMask(dswe_1990s.select('dswe').eq(0))
                .visualize(**visParams),
    description='DSWE_change_2010s_vs_1990s',
    folder='Okavango',
    fileNamePrefix='DSWE_change_2010s_vs_1990s',
    region=okavango,
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e13
)
task_change.start()


{'type': 'Image', 'bands': [{'id': 'dswe', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [8320, 7112], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, 21.791062667586118, 0, -0.00026949458523585647, -18.269307427723945]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [8320, 7112], 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, 21.791062667586118, 0, -0.00026949458523585647, -18.269307427723945]}], 'version': 1743630153062334, 'id': 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m/DSWE_1984_06', 'properties': {'date': '1984-06', 'month': '06', 'system:index': '0', 'year': '1984', 'system:footprint': {'type': 'LinearRing', 'coordinates': [[21.791197652440584, -18.26917221225489], [21.790913086913633, -18.269380951148047], [21.790909811862743, -20.186087935344936], [22.07143814001902, -20.186088130811537], [22.351678775395918, -20.186088123192537],

In [19]:
# Load DSWE ImageCollection
assets_dswe = ee.data.listAssets('projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m')['assets']

assets_dswe = list(filter(lambda asset: asset['type'] == 'IMAGE', assets_dswe))


asset_ids_dswe = [a['id'] for a in assets_dswe]

def process_dswe_image(id):
    year = ee.String(id).split('_').get(-2)
    month = ee.String(id).split('_').get(-1)
    img = ee.Image(id)
    binary = img.gt(0).toFloat().rename('dswe_bin')
    return img.addBands(binary).set({'year': year, 'month': month, 'date': ee.String(year).cat('-').cat(month)})

dswe_sentinel = ee.ImageCollection([process_dswe_image(id) for id in asset_ids_dswe])
print (dswe_sentinel.first().getInfo())
# Filters for year-specific and seasonal averages
dswe_compare = dswe_sentinel \
    .filter(ee.Filter.lt('month', '10')) \
    .filter(ee.Filter.gt('month', '06')) \
    .filter(ee.Filter.lt('year', '2025')) \



visParamsDSWE = { 'palette' : ['white', '#deebf7','#9ecae1','#3182bd'], 
              'min' :  0,  'max': 1}

visParams = {'palette' : ['#a6611a','#dfc27d','#f5f5f5','#80cdc1','#018571'], 
          'min' :  -1.,  'max' : 1}

task_change = ee.batch.Export.image.toDrive(
    image=dswe_sentinel.select('dswe').mean().selfMask()
                .visualize(**visParams),
    description='DSWE_sentinel',
    folder='Okavango',
    fileNamePrefix='DSWE_sentinel',
    region=okavango,
    scale=10,
    crs='EPSG:4326',
    maxPixels=1e13
)
task_change.start()


{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [24958, 21335], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 21.79124233064294, 0, -8.983152841195215e-05, -18.269307427723945]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [24958, 21335], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 21.79124233064294, 0, -8.983152841195215e-05, -18.269307427723945]}], 'version': 1744227861760223, 'id': 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m/DSWE_2018_12', 'properties': {'date': '2018-12', 'month': '12', 'year': '2018', 'system:index': '0', 'system:time_start': 1543622400000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[23.19249066546455, -20.18590816692056], [23.472731294750183, -20.185908204184578], [24.033308551541218, -20.185908134178124], [24.033307462662382, -18.269262310821514], [23.823032

In [20]:
# dswe_collection \
#     .filter(ee.Filter.eq('month', '01')) \
#     .filter(ee.Filter.eq('year', '1988')) \
#     .mean().getInfo()

In [21]:
okavango.getInfo()

{'type': 'Polygon',
 'coordinates': [[[21.696823393895034, -20.223198626985177],
   [24.102829253270034, -20.223198626985177],
   [24.102829253270034, -18.221176298743337],
   [21.696823393895034, -18.221176298743337],
   [21.696823393895034, -20.223198626985177]]]}

In [22]:
task_change.status()

{'state': 'READY',
 'description': 'DSWE_sentinel',
 'priority': 100,
 'creation_timestamp_ms': 1753208317024,
 'update_timestamp_ms': 1753208317024,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'UBT626GRMO5NY7RFU2KQ37NO',
 'name': 'projects/ee-okavango/operations/UBT626GRMO5NY7RFU2KQ37NO'}

In [31]:
import ee
ee.Initialize()

# List all current tasks (operations)
operations = ee.data.listOperations()
# operations
# Cancel tasks in READY or RUNNING state
for op in operations[:10]:
    state = op.get('metadata', {}).get('state', '')
    name = op.get('name', '')

    if state in ['READY', 'RUNNING', 'PENDING']:
        ee.data.cancelOperation(name)
        print(f"Canceled task: {name} ({state})")


Canceled task: projects/ee-okavango/operations/VEFI7F3ZXO4H36IOR66RFI5P (PENDING)
Canceled task: projects/ee-okavango/operations/MIFMWQDWRPUU6V7EAHLXX2T6 (PENDING)
Canceled task: projects/ee-okavango/operations/DM6OEAXSOGVISNE2Z7CJIGUJ (RUNNING)
Canceled task: projects/ee-okavango/operations/6KKU3WSO5XUZSQYAJPEMI26M (RUNNING)


In [29]:
import ee
# Initialize Earth Engine
ee.Initialize()

# Define visualization parameters
visParams = {
    'palette': ['white', '#deebf7', '#9ecae1', '#3182bd'],
    'min': 1,
    'max': 4
}

# Loop over years and months
for year in range(1985, 2025):
    for month in range(1, 13):
        month_str = f"{month:02d}"
        print (month_str,  str(year))
        # Filter by year and month
        filtered = dswe_collection \
            .filter(ee.Filter.eq('year', str(year))) \
            .filter(ee.Filter.eq('month', month_str)) \
            .mean()
        print (filtered.getInfo())
        image = filtered.select('dswe').unmask(0)\
                    .visualize(**visParams)

        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f'DSWE_{year}_{month_str}',
            folder='Okavango/monthly',
            fileNamePrefix=f'DSWE_{year}_{month_str}',
            region=okavango,
            scale=30,
            crs='EPSG:4326',
            maxPixels=1e13
        )
        # task.start()
        print(f"Started export for DSWE_{year}_{month_str}")


01 1985
{'type': 'Image', 'bands': []}
Started export for DSWE_1985_01
02 1985
{'type': 'Image', 'bands': []}
Started export for DSWE_1985_02
03 1985
{'type': 'Image', 'bands': []}
Started export for DSWE_1985_03
04 1985
{'type': 'Image', 'bands': [{'id': 'dswe', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Started export for DSWE_1985_04
05 1985


KeyboardInterrupt: 

In [32]:
import ee
# Initialize Earth Engine
ee.Initialize()

# Define visualization parameters
visParams = {
    'palette': ['white', '#deebf7', '#9ecae1', '#3182bd'],
    'min': 1,
    'max': 4
}

# Loop over years and months
for year in range(2018, 2025):
    for month in range(1, 13):
        month_str = f"{month:02d}"
        print (month_str,  str(year))
        # Filter by year and month
        filtered = dswe_sentinel \
            .filter(ee.Filter.eq('year', str(year))) \
            .filter(ee.Filter.eq('month', month_str)) \
            .mean()
        print (filtered.getInfo())
        image = filtered.select('DSWE').unmask(0)\
                    .visualize(**visParams)

        task = ee.batch.Export.image.toDrive(
            image=image,
            description=f'DSWE_{year}_{month_str}',
            folder='Okavango/monthly_sentinel',
            fileNamePrefix=f'DSWE_{year}_{month_str}',
            region=okavango,
            scale=10,
            crs='EPSG:4326',
            maxPixels=1e13
        )
        task.start()
        print(f"Started export for DSWE_{year}_{month_str}")


01 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_01
02 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_02
03 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_03
04 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_04
05 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_05
06 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_06
07 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_07
08 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_08
09 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_09
10 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_10
11 2018
{'type': 'Image', 'bands': []}
Started export for DSWE_2018_11
12 2018
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type

Started export for DSWE_2020_08
09 2020
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Started export for DSWE_2020_09
10 2020
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Started export for DSWE_2020_10
11 2020
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precisi

Started export for DSWE_2022_07
08 2022
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Started export for DSWE_2022_08
09 2022
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Started export for DSWE_2022_09
10 2022
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precisi

Started export for DSWE_2024_06
07 2024
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Started export for DSWE_2024_07
08 2024
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Started export for DSWE_2024_08
09 2024
{'type': 'Image', 'bands': [{'id': 'DSWE', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'dswe_bin', 'data_type': {'type': 'PixelType', 'precisi

In [8]:
# Load IL assets
def load_imagecollection(path, band_name=None):
    assets = ee.data.listAssets(path)['assets']
    ids = [a['id'] for a in assets]
    def load_img(id):
        img = ee.Image(id)
        if band_name:
            img = img.rename(band_name)
        year = ee.String(id).split('_').get(-1)
        return img.set({'year': year})
    return ee.ImageCollection([load_img(i) for i in ids])

IL_Collection = load_imagecollection('projects/ee-okavango/assets/Inman_Lyons/FloodArea')
IL_og = load_imagecollection('projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps', band_name='SR_B7')
IL_merged = IL_og.filter(ee.Filter.lt('year', '2019')).merge(IL_Collection)


# brown - green
visParams = {
    'palette': ['#a6611a', '#dfc27d', '#f5f5f5', '#80cdc1', '#018571'],
    'min': -1,
    'max': 1
}

# blues only
visParamsDSWE = {
    'palette': ['white', '#deebf7', '#9ecae1', '#3182bd'],
    'min': 0,
    'max': 1
}


visParamsDiff = {
    'palette': ['#ef8a62','#f7f7f7','#67a9cf'],
    'min': -1,
    'max': 1
}




### IL visuals

In [62]:


year_ranges = [
    {'name': '1985_2000_vs_2005_2020', 'earlyStart': '1985', 'earlyEnd': '2000', 'lateStart': '2005', 'lateEnd': '2020'},
    {'name': '1990_2005_vs_2010_2025', 'earlyStart': '1990', 'earlyEnd': '2005', 'lateStart': '2010', 'lateEnd': '2025'},
    {'name': '1990_1995_vs_2020_2025', 'earlyStart': '1990', 'earlyEnd': '1995', 'lateStart': '2020', 'lateEnd': '2025'},
    {'name': '2000_2005_vs_2020_2025', 'earlyStart': '1990', 'earlyEnd': '2005', 'lateStart': '2020', 'lateEnd': '2025'}

]

for r in year_ranges:
    early = IL_merged \
        .filter(ee.Filter.gte('year', r['earlyStart'])) \
        .filter(ee.Filter.lte('year', r['earlyEnd'])) \
        .mean()

    late = IL_merged \
        .filter(ee.Filter.gte('year', r['lateStart'])) \
        .filter(ee.Filter.lte('year', r['lateEnd'])) \
        .mean()

    change = late.subtract(early)

    # Export change visualized
    export_change = ee.batch.Export.image.toDrive(
        image=change.unmask(0).visualize(**visParams),
        description=f'IL_change_{r["name"]}',
        folder='Okavango',
        fileNamePrefix=f'IL_change_{r["name"]}',
        region=okavango,
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    export_change.start()

    # Export early visualized
    export_early = ee.batch.Export.image.toDrive(
        image=early.unmask(0).visualize(**visParamsDSWE),
        description=f'IL_{r["earlyStart"]}-{r["earlyEnd"]}',
        folder='Okavango',
        fileNamePrefix=f'IL_{r["earlyStart"]}-{r["earlyEnd"]}',
        region=okavango,
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    #export_early.start()

    # Export late visualized
    export_late = ee.batch.Export.image.toDrive(
        image=late.unmask(0).visualize(**visParamsDSWE),
        description=f'IL_{r["lateStart"]}-{r["lateEnd"]}',
        folder='Okavango',
        fileNamePrefix=f'IL_{r["lateStart"]}-{r["lateEnd"]}',
        region=okavango,
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    #export_late.start()



In [59]:
IL_mean = IL_merged.mean()
export_IL_mean = ee.batch.Export.image.toDrive(
    image=IL_mean.unmask(0).visualize(**visParams),
    description='IL_mean',
    folder='Okavango',
    fileNamePrefix='IL_mean',
    region=okavango,
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e13
)
# export_IL_mean.start()

IL_mean_diff = IL_mean.subtract(dswe_compare.select('dswe_bin').mean()).rename('IL_minus_DSWE_mean')

# Export each year and difference from 1990
years = ee.List.sequence(1990, 2024)

for y in years.getInfo():
    
    year_img = IL_merged.filter(ee.Filter.eq('year', str(y))).mean()

    export_year = ee.batch.Export.image.toDrive(
        image=year_img.unmask(0).clip(okavango).visualize(**visParamsDSWE),
        description=f'IL_{y}',
        folder='Okavango',
        fileNamePrefix=f'IL_{y}',
        region=okavango,
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    export_year.start()

    year_1990 = IL_merged.filter(ee.Filter.eq('year', '1990')).mean()
    diff_img = year_img.subtract(year_1990).rename('IL_diff_1990')

    export_diff = ee.batch.Export.image.toDrive(
        image=diff_img.unmask(0).visualize(**visParams),
        description=f'IL_diff_1990_{y}',
        folder='Okavango/IL_images',
        fileNamePrefix=f'IL_diff_1990_{y}',
        region=okavango,
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    export_diff.start()

    dswe_year = dswe_compare.filter(ee.Filter.eq('year', str(y))).mean()
    IL_minus_DSWE = year_img.subtract(dswe_year.select('dswe_bin')).rename('IL_minus_DSWE')  

  
    # purple means IL is higher
    export_IL_minus_DSWE = ee.batch.Export.image.toDrive(
      image=IL_minus_DSWE.unmask(0).visualize(**visParamsDiff),
      description='IL_minus_DSWE_' + str(y),
      folder='Okavango',
      fileNamePrefix='IL_minus_DSWE_year' + str(y),
      region=okavango,
      scale=30,
      crs='EPSG:4326',
      maxPixels=1e13
    )
    export_IL_minus_DSWE.start()
  
    export_IL_mean_diff = ee.batch.Export.image.toDrive(
        image=IL_mean_diff.unmask(0).visualize(**visParamsDiff),
        description='IL_minus_DSWE_mean',
        folder='Okavango',
        fileNamePrefix='IL_minus_DSWE_mean',
        region=okavango,
        scale=30,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    export_IL_mean_diff.start()

In [ ]:
# Compare IL and DSWE for year0
# IL_year = IL_merged.filter(ee.Filter.eq('year', year0)).mean()
# IL_mean = IL_merged.mean()

# IL_minus_DSWE = IL_year.subtract(dswe_year.select('dswe_bin')).rename('IL_minus_DSWE')
# IL_mean_diff = IL_mean.subtract(dswe_compare.select('dswe_bin').mean()).rename('IL_minus_DSWE_mean')


In [ ]:

# Morphological operations
kernel = ee.Kernel.circle(radius=90, units='meters')
dilated = dswe_year.select('dswe_bin').focal_min(kernel=kernel, iterations=1)
eroded = dilated.focal_max(kernel=kernel, iterations=1)

export_dilated = ee.batch.Export.image.toDrive(
    image=dilated.unmask(0),
    description='dilated_dswe_year',
    folder='Okavango',
    fileNamePrefix='dilated_dswe_year',
    region=okavango,
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e13
)
export_dilated.start()

export_eroded = ee.batch.Export.image.toDrive(
    image=eroded.unmask(0),
    description='eroded_dswe_year',
    folder='Okavango',
    fileNamePrefix='eroded_dswe_year',
    region=okavango,
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e13
)
export_eroded.start()